# Chapter 7: Function Decorators and Closures

The goal of this chapter is to explain how function decorators work. This requires understanding:

* How Python evaluates decorator syntax

* How Python decides whether a variable is local

* Why closures exist and how they work

* What problem is solved by `nonlocal`

This is helpful when tackling further decorator topics, such as:

* Implementing a well-behaved decorator

* Interesting decorators in the standard library

* Implementing a parameterised decorator

## Decorators 101

A decorator is a callable that takes another function as an argument (the decorated function).

The decorator may perform some processing with the decorated function, and returns it or replaces it with another function or callable object.

In [1]:
# Example 7-1. A decorater usually replaces a function with a different one.
def deco(func):
    def inner():
        print("running inner()")
    return inner

@deco
def target():
    print("running target()")
    
target()

running inner()


## When Python Executes Decorators

A key feature of decorators is that they run right after the decorated function is defined.

This is usually at *import time* (i.e. when a module is loaded by Python).

In [3]:
# Example 7-2. The registration.py module.
registery = []

def register(func):
    print("running register(%s)" % func)
    registery.append(func)
    return func

@register
def f1():
    print("running f1()")
    
@register
def f2():
    print("running f2()")
    
def f3():
    print("running f3()")
    
def main():
    print("running main()")
    print("registery ->", registery)
    f1()
    f2()
    f3()
    
if __name__ == "__main__":
    main()
    

running register(<function f1 at 0x1082d65e0>)
running register(<function f2 at 0x1082d68b0>)
running main()
registery -> [<function f1 at 0x1082d65e0>, <function f2 at 0x1082d68b0>]
running f1()
running f2()
running f3()


Note that `register` runs twice before any other function in the module. 

When `register` is called, it receives as an argument the function object being decorated.

After the module is loaded, the `registery` holds references to the two decorated functions.

These functions are only executed when explicitly called by `main`.

The above example is unusual in two ways:

1. The decorated function is defined in the same module as the decorated function. A real decorator is usually defined in one module and applied to functions in another.

2. The `register` decorator returns the same functions passed as an argument. In practice, most decorators define an inner function and return it.

If *registration.py* is imported (and not run as a script), the output is:

In [7]:
from examples import registration

running register(<function f1 at 0x10a6a21f0>)
running register(<function f2 at 0x10a7ec3a0>)


In [9]:
registration.registery

[<function examples.registration.f1()>, <function examples.registration.f2()>]

The main point here is to emphasize that function decorators are run as soon as the module is imported, but the decorated functions only run when explicitly invoked (*import time* vs. *runtime*).

## Decorator-Enhanced Strategy Pattern

Decorators can be used as a good enhancement to the [Strategy](https://en.wikipedia.org/wiki/Strategy_pattern) pattern presented in the previous chapter.

This solution has several advantages compared to those presented in the previous chapter (**p. 174**):

* Strategy functions don't need special names (i.e. drop `_promo` suffix).

* `@promotion` highlights the purpose of the decorated function and makes it easy to disable a promotion (just comment out the decorator).

* Discount strategies may be implemented in other modules, anywhere on the system, as long as the `@promotion` decorator is applied to them.

In [10]:
# Example 7-3. The promos list is filled by the promotion decorator
promos = []

def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity(order):
    """5% discount for customers with 1000 or more fidelity points."""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0


@promotion
def bulk_item(order):
    """10% discount for line orders with 20 or more units."""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount


@promotion
def large_order(order):
    "7% discount on orders with at least 10 distinct items."
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10 :
        return order.total() * .07
    else:
        return 0


def best_promo(order):
    """Select best discount available."""
    return max(promo(order) for promo in promos)

## Variable Scope Rules

Decorators almost always return inner functions.

Code the uses inner functions almost always depends on **closures** to operate correctly.

To understand closures, we need to look at how variable scopes work in Python.

In [2]:
# Example 7-5
b = 6

def f(a):
    print(a)
    print(b)
    b = 9
    
f(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

When Python compiles the body of the function, it decides that `b` is a local variable (because it is assigned within the function). This is why the function in **example 7-5** fails.

Python assumes that variables declared in the body of a function are local.

If we want the interpreter to treat `b` as a global variable in spite of assignment within the function, we need to use the `global` declaration.

In [3]:
# Example 7-6
# Example 7-5
b = 6

def f(a):
    global b
    print(a)
    print(b)
    b = 9
    
f(3)

3
6


In [4]:
b

9

## Closures

A closure is a function with an extended scope that encompasses nonglobal variables referenced in the body of the function but not defined there.

Consider the examples below:

* We call `Averager()` or `make_averager()` to get a callable object `avg` that will update the historical series and calculate the current mean.

* In **example 7-8** `avg` is an instance of `Averager()` and it is easy to see where `avg` keeps the history (`self.series` instance attribute).

* In **example 7-9** `avg` is the inner function of the higher-order function `make_averager`. In this case it is not obvious where the historical series is kept. 

In [19]:
# Example 7-8
class Averager():
    
    def __init__(self):
        self.series = []
    
    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total / len(self.series)

In [20]:
# Creates a callable instance
avg = Averager()
avg(10)

10.0

In [21]:
avg(11)

10.5

In [22]:
avg(12)

11.0

In [28]:
# Example 7-9
def make_averager():
    series = []
    
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total / len(series)
    
    return averager

In [29]:
# Returns the averager object
avg = make_averager()
avg(10)

10.0

In [30]:
avg(11)

10.5

In [31]:
avg(12)

11.0

`series` is a local variable of `make_averager`, but when `avg(10)` is called, `make_averager` has already returned and its local scope is gone.

Within `averager`, `series` is a *free variable*. This means that the variable is not bound in the local scope.

However, `averager` is a *closure* with access to its enclosing scope, which includes `series`.

![](https://www.codevoila.com/cvuploads/images/201606/python_function_and_python_closure.png)

Inspecting the `__code__` attribute of the returned function reveals how Python keeps the names of local and free variables in the compiled body of the function.

In [33]:
avg.__code__.co_varnames

('new_value', 'total')

In [34]:
avg.__code__.co_freevars

('series',)

In [40]:
avg.__closure__[0].cell_contents

[10, 11, 12]

## The `nonlocal` Declaration

In contrast, consider the following example which is meant to be a more efficient implementation of `averager` (saving only the numbers that are needed for calculation):

In [27]:
# Example 7-13. Broken implementation of make_averager.
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        count += 1
        total += new_value
        return total / count

    return averager

avg = make_averager()
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

This doesn't work because `count += 1` means the same as `count = count + 1` when `count` is a number or any immutable type.

With immutable types like numbers, strings, tuples, etc., rebinding them (as in `count = count + 1`) implicitly creates a new local variable `count`. It is no longer a free variable and therefore is not saved in the closure.

Since we are actually assigning to `count` in the body of `averager`, that makes it a local variable. The same applies to `total`.

To work around this we can use the `nonlocal` declaration, which lets us flag a variable as free variable even when it is assigned a new value within a function. If a new value is assigned to a `nonlocal` variable, the binding stored in the closure is changed.

In [41]:
# Example 7-14. Correct implementation of make_averager.
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count

    return averager

avg = make_averager()
avg(10)

10.0

## Implementing a Simple Decorator

In the following we implement a simple decorator that clocks every invocation of a function and prints the elapsed time, arguments passed and result of call.

In [6]:
# Example 7-15. A simple decorator to output the running time of functions

import time

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ", ".join(repr(arg) for arg in args)
        print("[%.8fs] %s(%s) -> %r" % (elapsed, name, arg_str, result))
        return result
    return clocked

@clock
def snooze(seconds):
    time.sleep(seconds)
    
@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n - 1)

In [2]:
snooze(.123)

[0.12738675s] snooze(0.123) -> None


In [3]:
factorial(6)

[0.00000039s] factorial(1) -> 1
[0.00004168s] factorial(2) -> 2
[0.00005773s] factorial(3) -> 6
[0.00009349s] factorial(4) -> 24
[0.00013207s] factorial(5) -> 120
[0.00032884s] factorial(6) -> 720


720

### How it works

Recall that 

```python
@clock
def factorial(n):
    return 1 if n < 2 else n * factorial(n - 1)
```

is the same as writing

```python
def factorial(n):
    return 1 if n < 2 else n * factorial(n - 1)

factorial = clock(factorial)
```

In both examples, `clock` gets the `factorial` function as its `func` argument and creates and returns the `clocked` function.

In [7]:
factorial.__name__

'clocked'

`factorial` now holds a reference to the `clocked` function. From now on, each time `factorial(n)` gets called `clocked(n)` gets executed. 

`clock` is a typical decorator; it replaces the decorated function with a new function that accepts the same arguments and (usually) returns whatever the decorator function was supposed to return, while also doing som extra processing.

However, `clock` has some shortcomings. It does not support keyword arguments, and it masks the `__name__` and `__doc__` of the decorated function.

`functools.wraps` is a ready-to-use decorator from the standard library. It copies the relevant attributes from `func` to `clock`. In the example below we also add proper support for keyword arguments.

In [9]:
# Example 7-17. An improved decorator

import time
import functools

def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.time()
        result = func(*args, **kwargs)
        elapsed = time.time() - t0
        name = func.__name__
        arg_lst = []
        if args:
            arg_lst.append(", ".join(repr(arg) for arg in args))
        if kwargs:
            pairs = ["%s=%r" % (k, v) for k, v in sorted(kwargs.items())]
            arg_lst.append(", ".join(pairs))
        arg_str = ", ".join(arg_lst)
        print("[%.8fs] %s(%s) - > %r" % (elapsed, name, arg_str, result))
        return result
    return clocked


In [15]:
@clock
def factorial(n):
    return 1 if n < 2 else n * factorial(n - 1)

_ = factorial(6)

[0.00000000s] factorial(1) - > 1
[0.00005198s] factorial(2) - > 2
[0.00007296s] factorial(3) - > 6
[0.00008917s] factorial(4) - > 24
[0.00010395s] factorial(5) - > 120
[0.00012326s] factorial(6) - > 720


In [16]:
@clock
def stupid(a, b, c=5, d="string"):
    result = a + b + c
    return str(result) + " " + d

_ = stupid(a=2, b=6, c=8, d="test")

[0.00000191s] stupid(a=2, b=6, c=8, d='test') - > '16 test'


## Decorators in the Standard Library

Python has three built-in decorators that are designed to decorate methods:

1. `property` (discussed in [chapter 19]())

2. `classmethod` (discussed in [chapter 9]())

3. `staticmethod` (discussed in [chapter 9]())

`functools.wraps` is a helper for building well-behaved decorators.

Two of the most interesting built-in decorators in the standard library are

1. `functools.lru_cache`

2. `functools.singledispatch`

### Memoization with `functools.lru_cache`

`lru_cache` implement memoization, a very efficient optimization technique that works by saving the results of previous invocations of an expensive function, avoiding repeat computations on previously used arguments.

The letters *LRU* stand for *Least Recently Used*, meaning that the growth of the cache is limited by discarding the entries that have not been read for a while.

In [25]:
%%time
# Example 7-18. Costly recursive way to compute numbers in Fibonacci series

@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 2) + fibonacci(n - 1)

_ = fibonacci(20)

[0.00000000s] fibonacci(0) - > 0
[0.00000095s] fibonacci(1) - > 1
[0.00041485s] fibonacci(2) - > 1
[0.00000000s] fibonacci(1) - > 1
[0.00000000s] fibonacci(0) - > 0
[0.00000000s] fibonacci(1) - > 1
[0.00004911s] fibonacci(2) - > 1
[0.00009298s] fibonacci(3) - > 2
[0.00055003s] fibonacci(4) - > 3
[0.00000000s] fibonacci(1) - > 1
[0.00000119s] fibonacci(0) - > 0
[0.00000095s] fibonacci(1) - > 1
[0.00006104s] fibonacci(2) - > 1
[0.00011587s] fibonacci(3) - > 2
[0.00000000s] fibonacci(0) - > 0
[0.00000000s] fibonacci(1) - > 1
[0.00021410s] fibonacci(2) - > 1
[0.00000095s] fibonacci(1) - > 1
[0.00000000s] fibonacci(0) - > 0
[0.00000072s] fibonacci(1) - > 1
[0.00005627s] fibonacci(2) - > 1
[0.00013590s] fibonacci(3) - > 2
[0.00044417s] fibonacci(4) - > 3
[0.00060582s] fibonacci(5) - > 5
[0.00123692s] fibonacci(6) - > 8
[0.00000095s] fibonacci(1) - > 1
[0.00000000s] fibonacci(0) - > 0
[0.00000095s] fibonacci(1) - > 1
[0.00004673s] fibonacci(2) - > 1
[0.00010610s] fibonacci(3) - > 2
[0.0000009

[0.00011182s] fibonacci(3) - > 2
[0.05508804s] fibonacci(4) - > 3
[0.05525398s] fibonacci(5) - > 5
[0.00000095s] fibonacci(0) - > 0
[0.00000000s] fibonacci(1) - > 1
[0.00004792s] fibonacci(2) - > 1
[0.00000119s] fibonacci(1) - > 1
[0.00000000s] fibonacci(0) - > 0
[0.00000000s] fibonacci(1) - > 1
[0.00003886s] fibonacci(2) - > 1
[0.00007725s] fibonacci(3) - > 2
[0.00016308s] fibonacci(4) - > 3
[0.00000095s] fibonacci(1) - > 1
[0.00000000s] fibonacci(0) - > 0
[0.00000095s] fibonacci(1) - > 1
[0.00042701s] fibonacci(2) - > 1
[0.00047469s] fibonacci(3) - > 2
[0.00000095s] fibonacci(0) - > 0
[0.00000000s] fibonacci(1) - > 1
[0.00006700s] fibonacci(2) - > 1
[0.00000000s] fibonacci(1) - > 1
[0.00000000s] fibonacci(0) - > 0
[0.00000119s] fibonacci(1) - > 1
[0.00007033s] fibonacci(2) - > 1
[0.00011683s] fibonacci(3) - > 2
[0.00025702s] fibonacci(4) - > 3
[0.00078392s] fibonacci(5) - > 5
[0.00098991s] fibonacci(6) - > 8
[0.05629015s] fibonacci(7) - > 13
[0.05698705s] fibonacci(8) - > 21
[0.00000

[0.05356479s] fibonacci(3) - > 2
[0.00000000s] fibonacci(0) - > 0
[0.00000095s] fibonacci(1) - > 1
[0.00004601s] fibonacci(2) - > 1
[0.00000095s] fibonacci(1) - > 1
[0.00000095s] fibonacci(0) - > 0
[0.00000095s] fibonacci(1) - > 1
[0.00004292s] fibonacci(2) - > 1
[0.00009704s] fibonacci(3) - > 2
[0.00020194s] fibonacci(4) - > 3
[0.05383515s] fibonacci(5) - > 5
[0.00000000s] fibonacci(0) - > 0
[0.00000000s] fibonacci(1) - > 1
[0.00003719s] fibonacci(2) - > 1
[0.00000000s] fibonacci(1) - > 1
[0.00000095s] fibonacci(0) - > 0
[0.00000095s] fibonacci(1) - > 1
[0.00005484s] fibonacci(2) - > 1
[0.00013304s] fibonacci(3) - > 2
[0.00024819s] fibonacci(4) - > 3
[0.00000000s] fibonacci(1) - > 1
[0.00000000s] fibonacci(0) - > 0
[0.00000095s] fibonacci(1) - > 1
[0.00004411s] fibonacci(2) - > 1
[0.00008607s] fibonacci(3) - > 2
[0.00000000s] fibonacci(0) - > 0
[0.00000000s] fibonacci(1) - > 1
[0.00003982s] fibonacci(2) - > 1
[0.00000000s] fibonacci(1) - > 1
[0.00000119s] fibonacci(0) - > 0
[0.0000000

[0.00000000s] fibonacci(1) - > 1
[0.00000095s] fibonacci(0) - > 0
[0.00000095s] fibonacci(1) - > 1
[0.00005198s] fibonacci(2) - > 1
[0.00114799s] fibonacci(3) - > 2
[0.00000000s] fibonacci(0) - > 0
[0.00000095s] fibonacci(1) - > 1
[0.00003695s] fibonacci(2) - > 1
[0.00000095s] fibonacci(1) - > 1
[0.00000095s] fibonacci(0) - > 0
[0.00000095s] fibonacci(1) - > 1
[0.00005007s] fibonacci(2) - > 1
[0.00010800s] fibonacci(3) - > 2
[0.00019002s] fibonacci(4) - > 3
[0.00138021s] fibonacci(5) - > 5
[0.00163007s] fibonacci(6) - > 8
[0.00203705s] fibonacci(7) - > 13
[0.00336790s] fibonacci(8) - > 21
[0.00000000s] fibonacci(1) - > 1
[0.00000095s] fibonacci(0) - > 0
[0.00000000s] fibonacci(1) - > 1
[0.00004888s] fibonacci(2) - > 1
[0.00009918s] fibonacci(3) - > 2
[0.00000000s] fibonacci(0) - > 0
[0.00000095s] fibonacci(1) - > 1
[0.00004888s] fibonacci(2) - > 1
[0.00000000s] fibonacci(1) - > 1
[0.00000072s] fibonacci(0) - > 0
[0.00000000s] fibonacci(1) - > 1
[0.00004888s] fibonacci(2) - > 1
[0.00009

[0.00000000s] fibonacci(0) - > 0
[0.00000095s] fibonacci(1) - > 1
[0.00009012s] fibonacci(2) - > 1
[0.04524899s] fibonacci(3) - > 2
[0.00000000s] fibonacci(0) - > 0
[0.00000000s] fibonacci(1) - > 1
[0.00004506s] fibonacci(2) - > 1
[0.00000000s] fibonacci(1) - > 1
[0.00000119s] fibonacci(0) - > 0
[0.00000000s] fibonacci(1) - > 1
[0.00006104s] fibonacci(2) - > 1
[0.00009799s] fibonacci(3) - > 2
[0.00017881s] fibonacci(4) - > 3
[0.04547906s] fibonacci(5) - > 5
[0.04572630s] fibonacci(6) - > 8
[0.04612684s] fibonacci(7) - > 13
[0.04698968s] fibonacci(8) - > 21
[0.00000000s] fibonacci(1) - > 1
[0.00000000s] fibonacci(0) - > 0
[0.00000000s] fibonacci(1) - > 1
[0.00005293s] fibonacci(2) - > 1
[0.00009489s] fibonacci(3) - > 2
[0.00000095s] fibonacci(0) - > 0
[0.00000000s] fibonacci(1) - > 1
[0.00010300s] fibonacci(2) - > 1
[0.00000000s] fibonacci(1) - > 1
[0.00000000s] fibonacci(0) - > 0
[0.00000000s] fibonacci(1) - > 1
[0.00005007s] fibonacci(2) - > 1
[0.00011110s] fibonacci(3) - > 2
[0.00027

...

[0.00020409s] fibonacci(4) - > 3
[0.00000119s] fibonacci(1) - > 1
[0.00000119s] fibonacci(0) - > 0
[0.00000000s] fibonacci(1) - > 1
[0.00004196s] fibonacci(2) - > 1
[0.00010180s] fibonacci(3) - > 2
[0.00000095s] fibonacci(0) - > 0
[0.00000000s] fibonacci(1) - > 1
[0.00006080s] fibonacci(2) - > 1
[0.00000000s] fibonacci(1) - > 1
[0.00000000s] fibonacci(0) - > 0
[0.00000095s] fibonacci(1) - > 1
[0.00004816s] fibonacci(2) - > 1
[0.00010610s] fibonacci(3) - > 2
[0.00029278s] fibonacci(4) - > 3
[0.00043797s] fibonacci(5) - > 5
[0.04554391s] fibonacci(6) - > 8
[0.04618812s] fibonacci(7) - > 13
[0.00000119s] fibonacci(0) - > 0
[0.00000095s] fibonacci(1) - > 1
[0.00005007s] fibonacci(2) - > 1
[0.00000000s] fibonacci(1) - > 1
[0.00000000s] fibonacci(0) - > 0
[0.00000095s] fibonacci(1) - > 1
[0.00004983s] fibonacci(2) - > 1
[0.00009823s] fibonacci(3) - > 2
[0.00019717s] fibonacci(4) - > 3
[0.00000095s] fibonacci(1) - > 1
[0.00000000s] fibonacci(0) - > 0
[0.00000095s] fibonacci(1) - > 1
[0.000050

[0.04385209s] fibonacci(4) - > 3
[0.00000000s] fibonacci(1) - > 1
[0.00000000s] fibonacci(0) - > 0
[0.00000000s] fibonacci(1) - > 1
[0.00005698s] fibonacci(2) - > 1
[0.00010538s] fibonacci(3) - > 2
[0.00000072s] fibonacci(0) - > 0
[0.00000119s] fibonacci(1) - > 1
[0.00008106s] fibonacci(2) - > 1
[0.00000095s] fibonacci(1) - > 1
[0.00000095s] fibonacci(0) - > 0
[0.00000000s] fibonacci(1) - > 1
[0.00005078s] fibonacci(2) - > 1
[0.00010109s] fibonacci(3) - > 2
[0.00023484s] fibonacci(4) - > 3
[0.00042772s] fibonacci(5) - > 5
[0.04435301s] fibonacci(6) - > 8
[0.00000095s] fibonacci(1) - > 1
[0.00000095s] fibonacci(0) - > 0
[0.00000000s] fibonacci(1) - > 1
[0.00004911s] fibonacci(2) - > 1
[0.00012064s] fibonacci(3) - > 2
[0.00000000s] fibonacci(0) - > 0
[0.00000000s] fibonacci(1) - > 1
[0.00005102s] fibonacci(2) - > 1
[0.00000119s] fibonacci(1) - > 1
[0.00000095s] fibonacci(0) - > 0
[0.00000000s] fibonacci(1) - > 1
[0.00004005s] fibonacci(2) - > 1
[0.00007725s] fibonacci(3) - > 2
[0.0001690

[0.00000000s] fibonacci(1) - > 1
[0.04361010s] fibonacci(2) - > 1
[0.04367185s] fibonacci(3) - > 2
[0.04441881s] fibonacci(4) - > 3
[0.04458594s] fibonacci(5) - > 5
[0.00000095s] fibonacci(0) - > 0
[0.00000095s] fibonacci(1) - > 1
[0.00004196s] fibonacci(2) - > 1
[0.00000095s] fibonacci(1) - > 1
[0.00000000s] fibonacci(0) - > 0
[0.00000072s] fibonacci(1) - > 1
[0.00006509s] fibonacci(2) - > 1
[0.00010300s] fibonacci(3) - > 2
[0.00019217s] fibonacci(4) - > 3
[0.00000000s] fibonacci(1) - > 1
[0.00000095s] fibonacci(0) - > 0
[0.00000000s] fibonacci(1) - > 1
[0.00003695s] fibonacci(2) - > 1
[0.00008416s] fibonacci(3) - > 2
[0.00000000s] fibonacci(0) - > 0
[0.00000000s] fibonacci(1) - > 1
[0.00004411s] fibonacci(2) - > 1
[0.00000000s] fibonacci(1) - > 1
[0.00000119s] fibonacci(0) - > 0
[0.00000000s] fibonacci(1) - > 1
[0.00006604s] fibonacci(2) - > 1
[0.00011182s] fibonacci(3) - > 2
[0.00019407s] fibonacci(4) - > 3
[0.00032306s] fibonacci(5) - > 5
[0.00056100s] fibonacci(6) - > 8
[0.0451891

[0.04361701s] fibonacci(3) - > 2
[0.04372978s] fibonacci(4) - > 3
[0.04395700s] fibonacci(5) - > 5
[0.04480481s] fibonacci(6) - > 8
[0.04522896s] fibonacci(7) - > 13
[0.04584026s] fibonacci(8) - > 21
[0.04746819s] fibonacci(9) - > 34
[0.00000119s] fibonacci(0) - > 0
[0.00000000s] fibonacci(1) - > 1
[0.00004315s] fibonacci(2) - > 1
[0.00000000s] fibonacci(1) - > 1
[0.00000119s] fibonacci(0) - > 0
[0.00000000s] fibonacci(1) - > 1
[0.00005412s] fibonacci(2) - > 1
[0.00010896s] fibonacci(3) - > 2
[0.00019503s] fibonacci(4) - > 3
[0.00000095s] fibonacci(1) - > 1
[0.00000095s] fibonacci(0) - > 0
[0.00000095s] fibonacci(1) - > 1
[0.00077105s] fibonacci(2) - > 1
[0.00081897s] fibonacci(3) - > 2
[0.00000095s] fibonacci(0) - > 0
[0.00000000s] fibonacci(1) - > 1
[0.00004292s] fibonacci(2) - > 1
[0.00000000s] fibonacci(1) - > 1
[0.00000000s] fibonacci(0) - > 0
[0.00000000s] fibonacci(1) - > 1
[0.00003982s] fibonacci(2) - > 1
[0.00008202s] fibonacci(3) - > 2
[0.00017309s] fibonacci(4) - > 3
[0.0010

[0.00000119s] fibonacci(0) - > 0
[0.00000119s] fibonacci(1) - > 1
[0.04268312s] fibonacci(2) - > 1
[0.04272795s] fibonacci(3) - > 2
[0.04282284s] fibonacci(4) - > 3
[0.04296613s] fibonacci(5) - > 5
[0.04320192s] fibonacci(6) - > 8
[0.00000072s] fibonacci(1) - > 1
[0.00000095s] fibonacci(0) - > 0
[0.00000000s] fibonacci(1) - > 1
[0.00003600s] fibonacci(2) - > 1
[0.00023890s] fibonacci(3) - > 2
[0.00000000s] fibonacci(0) - > 0
[0.00000095s] fibonacci(1) - > 1
[0.00004077s] fibonacci(2) - > 1
[0.00000000s] fibonacci(1) - > 1
[0.00000000s] fibonacci(0) - > 0
[0.00000000s] fibonacci(1) - > 1
[0.00003815s] fibonacci(2) - > 1
[0.00007725s] fibonacci(3) - > 2
[0.00015688s] fibonacci(4) - > 3
[0.00043988s] fibonacci(5) - > 5
[0.00000000s] fibonacci(0) - > 0
[0.00000000s] fibonacci(1) - > 1
[0.00004387s] fibonacci(2) - > 1
[0.00000119s] fibonacci(1) - > 1
[0.00000000s] fibonacci(0) - > 0
[0.00000119s] fibonacci(1) - > 1
[0.00004196s] fibonacci(2) - > 1
[0.00009513s] fibonacci(3) - > 2
[0.0001740

In [29]:
%%time
# Example 7-19. Faster implementation using caching

import functools

@functools.lru_cache()
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 2) + fibonacci(n - 1)

_ = fibonacci(20)

[0.00000000s] fibonacci(0) - > 0
[0.00000095s] fibonacci(1) - > 1
[0.00010276s] fibonacci(2) - > 1
[0.00000095s] fibonacci(3) - > 2
[0.00015807s] fibonacci(4) - > 3
[0.00000167s] fibonacci(5) - > 5
[0.00050926s] fibonacci(6) - > 8
[0.00000191s] fibonacci(7) - > 13
[0.00061417s] fibonacci(8) - > 21
[0.00000095s] fibonacci(9) - > 34
[0.00066257s] fibonacci(10) - > 55
[0.00000119s] fibonacci(11) - > 89
[0.00071096s] fibonacci(12) - > 144
[0.00000095s] fibonacci(13) - > 233
[0.00075603s] fibonacci(14) - > 377
[0.00000000s] fibonacci(15) - > 610
[0.00080681s] fibonacci(16) - > 987
[0.00000119s] fibonacci(17) - > 1597
[0.00085187s] fibonacci(18) - > 2584
[0.00000119s] fibonacci(19) - > 4181
[0.00089693s] fibonacci(20) - > 6765
CPU times: user 937 µs, sys: 667 µs, total: 1.6 ms
Wall time: 960 µs


Note that `functools.lru_cache` must be invoked as a regular function. This is because it accepts parameters. Its full signature is

`functools.lru_cache(maxsize=128, typed=False)`

The `maxsize` argument determins how many call results are stored.

After the cache is full, older results are discarded to make room (for optimal performance, set to a power of 2).

The `typed` argument, if set to `True`, stores results of different argument types separately.

Because `lru_cache` uses a `dict` to store the results, and the keys are made from the positional and keyword arguments used in the call, all the arguments taken by the decorated function *must be hashable*.

### Generic Functions with Single Dispatch

If a plain function is decorated with  `@singledispatch` it becomes a *generic function*: a group of functions to perform the same operation in different ways, depending on the type of the first argument.

`@singledispatch` lets you easily provide a specialised function for classes that you can't even edit.

For functions annotated with types, the decorator will infer the type of the first argument automatically. If not, they are required to be passed as an argument.

Note that the name of the decorated functions are irrelevant, hence the use of underscore.

A notable quality of `@singledispatch` is that you can register specialised functions anywhere in the system, in any module. If you later add a module with a new user-defined type, simply provide a new custom function to handle that type.

**Note:** When possible, register the specialised functions to handle abstract base classes (such as `numbers.Integral` a superclass of `int`) instead of concrete implementations (such as `int`). This will allow the function to handle a wider variety of compatible types.

In [43]:
# Example from official Python docs: https://docs.python.org/3/library/functools.html

from functools import singledispatch

@singledispatch
def fun(arg, verbose=False):
    if verbose:
        print("Let me just say,", end=" ")
        print(arg)

@fun.register(int)    
def _int(arg, verbose=False):
    if verbose:
        print("Strength in numbers, eh?", end=" ")
    print(arg)

@fun.register(list)
def _lst(arg, verbose=False):
    if verbose:
        print("Enumerate this:")
    for i, elem in enumerate(arg):
        print(i, elem)

fun(5, verbose=True)

Strength in numbers, eh? 5


In [44]:
fun(["5"], verbose=True)

Enumerate this:
0 5


In [45]:
fun("5", verbose=True)

Let me just say, 5


## Stacked Decorators

Since decorators are functions they can be composed (as in example 7-19).

When to decorators `@d1` and `@d2` are applied to a function `f` in that order, the result is the same as `d1(d2(f))`.

## Parameterised Decorators

When parsing a decorator in source code, Python takes the decorated function and passes it as the first argument to the decorator function.

To make the decorator accept other inputs, we must make a *decorator factory* that takes those arguments and returns a decorator, which is the applied to the function to be decorated.

We illustrate this by revisiting example 7-2.

In [46]:
# Example 7-2. The registration.py module.
registery = []


def register(func):
    print("running register(%s)" % func)
    registery.append(func)
    return func


@register
def f1():
    print("running f1()")


@register
def f2():
    print("running f2()")


def f3():
    print("running f3()")


if __name__ == "__main__":
    print("running main()")
    print("registery ->", registery)
    f1()
    f2()
    f3()

running register(<function f1 at 0x110a95680>)
running register(<function f2 at 0x110a954d0>)
running main()
registery -> [<function f1 at 0x110a95680>, <function f2 at 0x110a954d0>]
running f1()
running f2()
running f3()


Conceptually, the new decorator in the example below is not a decorator but a decorator factory. When called, it returns the actual decorator that will be applied to the target function.

In [48]:
# Example 7-22. To accept parameters, the new register decorator must be called as a function.

# Now a set, makes adding and discarding faster
registery = set()


# register now takes optional keyword argument
def register(active=True):
    # Because decorate is a decorator, it must return the function
    def decorator(func):
        print("running register(active=%s) -> decorate(%s)" % (active, func))
        if active:
            registery.add(func)
        else:
            registery.discard(func)
        return func
    # register is our decorator factory, so it returns the decorator
    return decorator


# register must be invoked as a function
@register(active=False)
def f1():
    print("running f1()")


@register()
def f2():
    print("running f2()")


def f3():
    print("running f3()")


if __name__ == "__main__":
    print("running main()")
    print("registery ->", registery)
    f1()
    f2()
    f3()

running register(active=False) -> decorate(<function f1 at 0x110a95710>)
running register(active=True) -> decorate(<function f2 at 0x110a95b90>)
running main()
registery -> {<function f2 at 0x110a95b90>}
running f1()
running f2()
running f3()


### The Parameterised Clock Decorator

In this section we add a feature to the `clock` decorator that allows user to pass a format string to control the output of the decorated function.

In [51]:
# Example 7-25. Parameterised clock decorator

import time

DEFAULT_FMT = "[{elapsed:.8f}s] {name}({args}) -> {result}"

# clock is parameterised decorator factory
def clock(fmt=DEFAULT_FMT):
    # decorate is actual decorater
    def decorate(func):
        # clocked wraps the decorated function
        def clocked(*_args):
            t0 = time.time()
            # _result is actual result of decorated function
            _result = func(*_args)
            elapsed = time.time() - t0
            name = func.__name__
            args= ", ".join(repr(arg) for arg in _args)
            result = repr(_result)
            # using locals() here allows any local variable of clocked to be referenced by fmt
            print(fmt.format(**locals()))
            return _result
        return clocked
    return decorate


@clock()
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

[0.12332010s] snooze(0.123) -> None
[0.12521029s] snooze(0.123) -> None
[0.12704778s] snooze(0.123) -> None


In [59]:
# Example 7-26. Alternative string format

@clock("{name}: {elapsed} seconds")
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

snooze: 0.12532281875610352 seconds
snooze: 0.12805700302124023 seconds
snooze: 0.12776708602905273 seconds


In [61]:
# Example 7-28. Alternative string format

@clock("{name}({args}) dt={elapsed:.3f}s")
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

snooze(0.123) dt=0.128s
snooze(0.123) dt=0.126s
snooze(0.123) dt=0.126s
